# PIP INSTALLS

In [1]:
!pip install psycopg2-binary

     |████████████████████████████████| 2.9MB 2.8MB/s 


In [2]:
!pip install cbpro

     |████████████████████████████████| 204kB 8.3MB/s 
     |████████████████████████████████| 593kB 42.6MB/s 
     |████████████████████████████████| 368kB 40.7MB/s 
  Created wheel for websocket-client: filename=websocket_client-0.40.0-py2.py3-none-any.whl size=198284 sha256=e63a57f5a4e9c33afdf679824827151dde9255fd1cb918010ab3a7e725d9db07
  Stored in directory: /root/.cache/pip/wheels/b0/d2/e2/8275906d09421f074dc2d05381643a0b4b3c4945d845b14bf9
Successfully built websocket-client
ERROR: tfds-nightly 1.2.0.dev201909050105 has requirement requests>=2.19.0, but you'll have requests 2.13.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.21.0, but you'll have requests 2.13.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement six~=1.12.0, but you'll have six 1.10.0 which is incompatible.
ERROR: google-api-core 1.14.2 has requirement requests<3.0.0dev,>=2.18.0, but you'll have requests 2.13.0 which is incompatible.
ERROR: datascience 0.10.6 has requ

# Importing Libraries

In [0]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import MetaData, Table
from sqlalchemy import Column, String, Integer
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import psycopg2 as ps

import requests
import json

import cbpro
import time
import datetime

# CB Pro

In [8]:
# connect to public client
public_client = cbpro.PublicClient()

# get ohlc data - other parameters (start, end)
orders = public_client.get_product_historic_rates('ETH-USD', granularity=60)
order_candle = orders[0]

# function to convert time to datetime
def to_datetime(epoch_time):
  dt_time = datetime.datetime.utcfromtimestamp(epoch_time).strftime('%Y-%m-%d %H:%M:%S')
  return dt_time

print('current time:', to_datetime(time.time()), '\ncbpro request time:', to_datetime(order_candle[0]))

current time: 2019-10-15 15:36:10 
cbpro request time: 2019-10-15 15:28:00


In [0]:
# get current time using:
# time.time()

# check time on cbpro request
datetime.datetime.utcfromtimestamp(1570940504.8549929).strftime('%Y-%m-%d %H:%M:%S')

'2019-10-13 04:21:44'

# AWS Postgres DB

In [0]:
POSTGRES_ADDRESS = '' ## End Point
POSTGRES_PORT = '' ##Port
POSTGRES_USERNAME = '' ## CHANGE THIS TO YOUR PANOPLY/POSTGRES USERNAME
POSTGRES_PASSWORD = '' ## CHANGE THIS TO YOUR PANOPLY/POSTGRES PASSWORD 
POSTGRES_DBNAME = 'postgres' ## CHANGE THIS TO YOUR DATABASE NAME

# A long string that contains the necessary Postgres login information 
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME
               , password=POSTGRES_PASSWORD, ipaddress=POSTGRES_ADDRESS,port=POSTGRES_PORT, dbname=POSTGRES_DBNAME))

In [12]:
# check connection
try:
    conn = ps.connect(host=POSTGRES_ADDRESS,database=POSTGRES_DBNAME,user=POSTGRES_USERNAME,password=POSTGRES_PASSWORD,port=POSTGRES_PORT)
except ps.OperationalError as e:
    raise e
else:
    print('Connected!')

Connected!


In [16]:
# create table

conn = ps.connect(host=POSTGRES_ADDRESS,database=POSTGRES_DBNAME,user=POSTGRES_USERNAME,password=POSTGRES_PASSWORD,port=POSTGRES_PORT)

cur = conn.cursor()
cur.execute('''
  CREATE TABLE cbpro_test
  (
  trade_id integer primary key,
  price float,
  size float,
  time varchar(25),
  bid float,
  ask float,
  volume float
  )

''')

conn.commit()
print("Table created Successfully!")
conn.close()

Table created Successfully!


In [13]:
# check table names in database

conn = ps.connect(host=POSTGRES_ADDRESS,database=POSTGRES_DBNAME,user=POSTGRES_USERNAME,password=POSTGRES_PASSWORD,port=POSTGRES_PORT)

cur = conn.cursor()

cur.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")
for table in cur.fetchall():
    print(table)

('cbpro_test',)
('cbpro_test2',)
('cbpro_test3',)
('cbpro_test4',)


In [46]:
# insert data into table

conn = ps.connect(host=POSTGRES_ADDRESS,database=POSTGRES_DBNAME,user=POSTGRES_USERNAME,password=POSTGRES_PASSWORD,port=POSTGRES_PORT)

cur = conn.cursor()

insert_query = "INSERT INTO cbpro_test4 (time, open, high, low, close, volume) VALUES (%s, %s, %s, %s, %s, %s)"  
cur.execute(insert_query, cbpro_candle)

select_query = """SELECT * FROM cbpro_test4"""
cur.execute(select_query)
print(cur.fetchall())

conn.commit()
cur.close()

[(1571072820, 187.02, 187.36, 187.02, 187.22, 247.38168252), (1571072820, 187.02, 187.36, 187.02, 187.22, 247.38168252)]


In [15]:
# check data in table

conn = ps.connect(host=POSTGRES_ADDRESS,database=POSTGRES_DBNAME,user=POSTGRES_USERNAME,password=POSTGRES_PASSWORD,port=POSTGRES_PORT)

cur = conn.cursor()

select_query = """SELECT * FROM cbpro_test4"""
cur.execute(select_query)

for item in cur.fetchall():
  print(item)

(1571072820, 187.02, 187.36, 187.02, 187.22, 247.38168252)
(1571072820, 187.02, 187.36, 187.02, 187.22, 247.38168252)
(123, 187.02, 187.36, 187.02, 187.22, 247.38168252)
(6789, 187.02, 187.36, 187.02, 187.22, 247.38168252)
(6789, 187.02, 187.36, 187.02, 187.22, 247.38168252)
(6789, 187.02, 187.36, 187.02, 187.22, 247.38168252)
(12345, 187.02, 187.36, 187.02, 187.22, 247.38168252)
(1571072820, 187.02, 187.36, 187.02, 187.22, 247.38168252)
